# 강의 게시판 데이터 전처리

* 데이터 클리어링 : 스크래핑한 데이터를 데이터 분석에 용이하도록 정제합니다.

1. Object 타입의 변수 중 필요에 따라 숫자형 변수로 변경합니다.
2. 불필요한 텍스트 제거합니다.
3. 'course_info' 변수에 강의 섹션 개수('course_section')와 총 강의 시간('course_time_index') 데이터가 함께 스크래핑되어 있습니다. 섹션 개수와 총 강의 시간 변수를 만들어 각 변수에 맞게 데이터를 나누어 값을 기입하였습니다.

In [1]:
import pandas as pd
import math
import numpy as np
import re

pd.set_option('display.width', 1000)

In [2]:
course_df = pd.read_csv('o1_course.csv', index_col = 0)

In [3]:
course_df['review_score'] = course_df['review_score'].str.replace(r'\D+', '', regex = True).astype(float) * 0.1
course_df['review_cnt'] = course_df['review_cnt'].str.replace('개의 수강평', '')
course_df['student_cnt'] = course_df['student_cnt'].str.replace('명', '')

course_df['price'] = course_df['price'].str.replace('원', '')
course_df['price'] = course_df['price'].str.replace(',', '')
course_df['price'] = course_df['price'].str.replace('무료', '0').astype(int)
course_df['discount_rate'] = course_df['discount_rate'].str.replace('%', '').astype(int)
course_df['sale_price'] = course_df['sale_price'].str.replace('원', '')
course_df['sale_price'] = course_df['sale_price'].str.replace(',', '')
course_df['sale_price'] = course_df['sale_price'].str.replace('무료', '0').astype(int)

course_df['course_info'] = course_df['course_info'].str.replace('\n', '')
course_df['course_section'] = course_df['course_info'].str[2:4]
course_df['course_section'] = course_df['course_section'].str.replace(r'\D+', '', regex = True)
course_df['course_time_index'] = course_df['course_info'].str.split('수업˙총 ').str[1]
course_df['course_time_index'] = course_df['course_time_index'].str.replace('수업', '0')
course_df['course_time_index'] = course_df['course_time_index'].str.strip()

course_df['limit_time'] = course_df['limit_time'].str.replace('\n', '')
course_df['limit_time'] = course_df['limit_time'].str.replace('수강기한: ', '')
course_df['limit_time'] = course_df['limit_time'].str.strip()

course_df['certificates'] = course_df['certificates'].str.replace('\n', '')
course_df['certificates'] = course_df['certificates'].str.replace('수료증 ', '')
course_df['certificates'] = course_df['certificates'].str.replace(' 강의', '')
course_df['certificates'] = course_df['certificates'].str.strip()

course_df['level'] = course_df['level'].str.replace(' 대상', '')

In [4]:
course_df = course_df.drop(['course_info'], axis = 1)

In [5]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1753 entries, 0 to 1752
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   main_category      1753 non-null   object 
 1   sub_category       1753 non-null   object 
 2   course_title       1753 non-null   object 
 3   review_score       1753 non-null   float64
 4   review_cnt         1753 non-null   object 
 5   student_cnt        1753 non-null   object 
 6   instructor         1753 non-null   object 
 7   related_tags       1753 non-null   object 
 8   price              1753 non-null   int32  
 9   discount_rate      1753 non-null   int32  
 10  sale_price         1753 non-null   int32  
 11  limit_time         1753 non-null   object 
 12  certificates       1753 non-null   object 
 13  level              1753 non-null   object 
 14  course_section     1753 non-null   object 
 15  course_time_index  1644 non-null   object 
dtypes: float64(1), int32(3),

In [6]:
course_df.describe(include ='all')

,main_category,sub_category,course_title,review_score,review_cnt,student_cnt,instructor,related_tags,price,discount_rate,sale_price,limit_time,certificates,level,course_section,course_time_index
count,1753,1753,1753,1753.000000,1753,1753,1753,1753,1.753000e+03,1753.000000,1.753000e+03,1753,1753,1753,1753,1644
unique,8,42,1753,NaN,220,1023,462,1101,NaN,NaN,NaN,12,2,3,97,829
top,개발 · 프로그래밍,프로그래밍 언어,모든 개발자의 실무를 위한 올인원 기본기 클래스,NaN,0,5,유용한IT학습,[['Python']],NaN,NaN,NaN,무제한,발급,초급,12,2시간 55분
freq,768,174,1,NaN,438,28,146,34,NaN,NaN,NaN,1349,1331,934,73,9
mean,NaN,NaN,NaN,3.519681,NaN,NaN,NaN,NaN,4.299155e+04,8.192812,3.916334e+04,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,2.058446,NaN,NaN,NaN,NaN,5.822112e+04,25.944981,5.842611e+04,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,1.300000,NaN,NaN,NaN,NaN,1.100000e+04,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,4.700000,NaN,NaN,NaN,NaN,3.300000e+04,0.000000,3.300000e+04,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,4.900000,NaN,NaN,NaN,NaN,5.500000e+04,0.000000,5.500000e+04,NaN,NaN,NaN,NaN,NaN


'course_time_index'(강의 시간)변수가 존재하지 않은 경우(강의가 동영상이 아닌 텍스트 혹은 ppt 자료인 경우)가 존재하므로 결측치를 0으로 처리하였습니다. 

In [10]:
course_df['course_time_index'] = course_df['course_time_index'].fillna(0)

'course_time_index'는 시간과 분이 혼합되어 있으므로 총 강의 시간을 파악하기 어렵습니다.
분석에 용이하도록 분을 기준으로 강의 시간을 통합한 변수('course_time_min')를 추가하였습니다.

In [13]:
hour_values = []
min_values = []
coures_time_min = []

for values in course_df['course_time_index'].values :
    if values == 0 :
        hour_values.append(0)
        min_values.append(0)
    elif ('시간' in values) and ('분' in values) :
        hour_values.append(values.split('시간')[0].strip())
        min_values.append(values.split('시간')[1].replace('분', '').strip())
    elif '시간' in values :
        hour_values.append(values.split('시간')[0].strip())
        min_values.append(0)
    elif '분' in values :
        hour_values.append(0)
        min_values.append(values.split('분')[0].strip())
    else :
        print("다른 구조가 있습니다.")

for hour, min in zip(hour_values, min_values) :
    coures_time_min.append((int(hour) * 60) + int(min))

course_df['course_time_min'] = coures_time_min

In [16]:
course_df.to_csv('o2_course.csv', encoding = 'utf-8-sig')